# SITCOM-1089 - Strong Vibration Analysis (VMS data)
 Analysis incomplete

[SITCOM-1089]: https://jira.lsstcorp.org/browse/SITCOM-1089

In [ ]:
import sys, time, os, asyncio
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy import signal
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
from tqdm import tqdm
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import (
    getEfdData,
    makeEfdClient,
    clipDataToEvent,
    calcNextDay,
)

%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import sitcom1089

In [ ]:
vms_m1m3_filename = "/home/p/pferguso/u/scratch/vms_data/M1M3-2023-06-28T00:00.hdf"

begin_time = Time("2023-06-28 01:08:00", format="iso", scale="utc")
end_time = Time("2023-06-28 01:20:00", format="iso", scale="utc")
raised_begin = Time("2023-06-28 01:08:15", format="iso", scale="utc")
raised_end = Time("2023-06-28 01:11:55", format="iso", scale="utc")
lowering_end = Time("2023-06-28 01:15:15", format="iso", scale="utc")
down_end = Time("2023-06-28 01:20:00", format="iso", scale="utc")

time_dict = {}
time_dict["total"] = {"begin": begin_time, "end": end_time}
time_dict["raised"] = {"begin": begin_time, "end": raised_end}
time_dict["lowering"] = {"begin": raised_end, "end": lowering_end}
time_dict["down"] = {"begin": lowering_end, "end": end_time}

In [ ]:
# client=makeEfdClient()
# efd_dict=get_efd_data(begin_time, end_time, client)

vms_m1m3_data = sitcom1089.vms_data_to_pandas(
    vms_m1m3_filename,
    vms_type="m1m3",
    begin_time=begin_time.unix,
    end_time=end_time.unix,
)

In [ ]:
vms_m1m3_data

In [ ]:
fig, ax = plt.subplots(1, dpi=125, sharex=True, figsize=(10, 3))
plt.suptitle(
    f"M1M3 VMS\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n",
    y=0.99,
)

for i in np.arange(3):
    ax.scatter(
        Time(vms_m1m3_data["times"], format="unix").datetime,
        vms_m1m3_data[f"total_{i+1}"] - 0.01 * i,
        label=f"total channel {i+1} \noffset = {- 0.002 * i:0.3f}",
        s=1,
    )
t1 = Time("2023-06-28 01:08:15", format="iso", scale="utc").datetime
t2 = Time("2023-06-28 01:08:25", format="iso", scale="utc").datetime
ax.set(
    xlabel="Time",
    ylabel="total acceleration [m/s$^2$]",
    ylim=(-0.02, 0.01),
    # xlim=(t1,t2)
)
ax.legend(ncol=3, loc=9, edgecolor="white")

In [ ]:
def log_linspace(begin, end, npoints):
    return 10 ** np.linspace(np.log10(begin), np.log10(end), npoints)

In [ ]:
psd_dict

In [ ]:
psd_dict = {}

In [ ]:
wtimes = [30e3]  # 5e2,1e3,1.5e3,2e3,3e3,5e3,10e3,15e3,20e3,25e3,30e3
for j, i in enumerate(wtimes):
    # print(i)
    # if i not in psd_dict.keys():
    w = (
        np.linspace(0.1, 100, int(i)) * 2 * np.pi
    )  # convert log_linspace(0.5, 40, int(i)) * 6.28
    x = Time(vms_m1m3_data["times"], format="unix").unix
    psd_df = pd.DataFrame({"frequency": w / (2 * np.pi)})
    for k in tqdm(range(3)):
        y = vms_m1m3_data[f"total_{k + 1}"]
        pgram = signal.lombscargle(x, y, w, normalize=True, precenter=True)
        psd_df[f"total_{k + 1}"] = pgram
        psd_df.to_csv("./1089_data/psd.csv", index=False)
        # psd_dict[f"total_{k + 1}"]={"x":w,"y":pgram}

In [ ]:
pd.DataFrame()

In [ ]:
for j,i in enumerate(wtimes):
    if i > 3e3:
        points_x=get_peak_points(w/6.28, pgram , height=0.0005, )
        plt.plot(psd_dict[i]["x"]/ 6.28, psd_dict[i]["y"] - 0.03 * j, label=i)#, s=0.1)
        print(i,points_x)
plt.legend()

In [ ]:
points_x=get_peak_points(w/6.28, pgram, height=0.0005)
plt.plot(w/ 6.28, pgram)#, s=0.1)
print(points_x)
#plt.yscale('log')
#plt.xscale('log')

In [ ]:
times=Time(efd_dict["ims"]["timestamp"], format="unix_tai", scale="utc").datetime
for key in ["xPosition","yPosition","zPosition"]:
    plt.plot(times, (efd_dict["ims"][key]- efd_dict["ims"][key].mean())* 1e3, label=key)

plt.axvline(time_dict["raised"]["end"].datetime, label="begin lowering", c="k", ls="dashdot")
plt.axvline(time_dict["lowering"]["end"].datetime, label="end lowering", c="k", ls="dotted")
plt.legend()
plt.xlabel("Time")
plt.ylabel("$\Delta$ postion [mm]")
plt.title(f"IMS Position\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}")

In [ ]:
times=Time(efd_dict["ims"]["timestamp"], format="unix_tai", scale="utc").datetime
for key in ["xRotation","yRotation","zRotation"]:
    plt.plot(times, np.rad2deg(efd_dict["ims"][key]- efd_dict["ims"][key].mean()), label=key)

plt.axvline(time_dict["raised"]["end"].datetime, label="begin lowering", c="k", ls="dashdot")
plt.axvline(time_dict["lowering"]["end"].datetime, label="end lowering", c="k", ls="dotted")
plt.legend()
plt.xlabel("Time")
plt.ylabel("$\Delta$ rotation [deg]")
plt.title(f"IMS Position\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}")

In [ ]:
times=Time(efd_dict["el"]["timestamp"], format="unix_tai", scale="utc").datetime
plt.figure(dpi=125, figsize=(12,3))
for key in ["actualTorque"]:
    plt.plot(times, np.rad2deg(efd_dict["el"][key]- efd_dict["el"][key].mean()), label=key, lw=0.3)

plt.axvline(time_dict["raised"]["end"].datetime, label="begin lowering", c="k", ls="dashdot")
plt.axvline(time_dict["lowering"]["end"].datetime, label="end lowering", c="k", ls="dotted")
plt.legend(ncol=3, loc=9)
plt.xlabel("Time")
plt.ylabel("$\Delta$ rotation [deg]")
plt.title(f"TMA actualTorque\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}")
plt.ylim(-2e7,3e7)

In [ ]:
dict_key="el"
key="actualTorque"
times=Time(efd_dict[dict_key]["timestamp"], format="unix_tai", scale="utc")
fs=1/np.mean(np.diff(times.unix))
plt.figure(dpi=125, figsize=(12,4))
for time_key in time_dict.keys():
    sel=(times > time_dict[time_key]["begin"])
    sel&=(times <= time_dict[time_key]["end"])
    vals=efd_dict[dict_key][key][sel]


    freq,psd=get_freq_psd(vals, 1/fs)
    yvals=pd.DataFrame({"psd":psd})["psd"].rolling(30).mean()
    plt.plot(freq,yvals, zorder=9, lw=1, label=time_key)
plt.yscale('log')
plt.ylim(1e11,1e17)
plt.legend()
plt.xlabel("Frequency (Hz)")
plt.ylabel("power")
plt.title(f"TMA Elevation {key}\nEvent {begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}")

In [ ]:
dict_key="ims"
fig, axs=plt.subplots(3,1,dpi=125, figsize=(14,10))
plt.suptitle(f"Event {begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}", y=0.92)
for i,key in enumerate(["xPosition","yPosition","zPosition"]):
    ax=axs[i]
    times=Time(efd_dict[dict_key]["timestamp"], format="unix_tai", scale="utc")
    fs=1/np.mean(np.diff(times.unix))
    for time_key in time_dict.keys():
        sel=(times > time_dict[time_key]["begin"])
        sel&=(times <= time_dict[time_key]["end"])
        vals=efd_dict[dict_key][key][sel]


        freq,psd=get_freq_psd(vals, 1/fs)
        yvals=pd.DataFrame({"psd":psd})["psd"].rolling(30).mean()
        ax.plot(freq,yvals, zorder=9, lw=1, label=time_key)
    title_str=f"IMS {key}"
    ax.set(yscale='log', xlabel="Frequency (Hz)",ylabel="power", title=title_str)
    #plt.ylim(1e11,1e17)
    ax.legend()
plt.subplots_adjust(hspace=0.4)

In [ ]:
dict_key="ims"
fig, axs=plt.subplots(3,1,dpi=125, figsize=(14,10))
plt.suptitle(f"Event {begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}", y=0.92)
for i,key in enumerate(["xRotation","yRotation","zRotation"]):
    ax=axs[i]
    times=Time(efd_dict[dict_key]["timestamp"], format="unix_tai", scale="utc")
    fs=1/np.mean(np.diff(times.unix))
    for time_key in time_dict.keys():
        sel=(times > time_dict[time_key]["begin"])
        sel&=(times <= time_dict[time_key]["end"])
        vals=efd_dict[dict_key][key][sel]


        freq,psd=get_freq_psd(vals, 1/fs)
        yvals=pd.DataFrame({"psd":psd})["psd"].rolling(30).mean()
        ax.plot(freq,yvals, zorder=9, lw=1, label=time_key)
    title_str=f"IMS {key}"
    ax.set(yscale='log', xlabel="Frequency (Hz)",ylabel="power", title=title_str)
    #plt.ylim(1e11,1e17)
    ax.legend()
plt.subplots_adjust(hspace=0.4)

In [ ]:
for i,key in enumerate([i for i in sorted(key_m1m3_dict.values()) if ("m1m3" in i)]):
    
    freq,psd=get_freq_psd(vms_m1m3_data[key], np.mean(np.diff(vms_m1m3_data["times"])))
    points_x=get_peak_points(freq, psd)
    if points_x is not None:
        points_y=np.ones_like(points_x) * i * step * -1
        print(f"{key}: {str([int(i) for i in points_x])[1:-1]} Hz")
    else: 
        print(f"{key} no peaks")
    plt.plot(freq,psd - i * step, label=key, zorder=9, lw=1)

In [ ]:
times

In [ ]:
fs=1/np.mean(np.diff(times.unix))
fs

In [ ]:
ydict={
    "el":"actualTorque",
    "ims":
}
fig,axs=plt.subplots(3, dpi=175, figsize=(10,4), sharex=True)
plt.suptitle(f"M1M3\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}", y=0.97)

for i, key in enumerate(["el","ims"])
times=Time(efd_dict["el"]["timestamp"], format="unix_tai", scale="utc")
fs=1/np.mean(np.diff(times.unix))
ax=axs[0]
ax.specgram(efd_dict["el"]["actualTorque"], Fs=fs, detrend="mean", vmin=-11)
ax.set(ylabel="Channel 1\nFreq [hz]", yticks=np.arange(0,16,25))

# ax=axs[1]
# ax.specgram(vms_m1m3_data["total_2"] - np.mean(vms_m1m3_data["total_2"]), Fs=fs, detrend="mean", vmin=-110)
# ax.set(ylabel="Channel 2\nFreq [hz]", yticks=np.arange(25,125,25))

# ax=axs[2]
# ax.specgram(vms_m1m3_data["total_3"] - np.mean(vms_m1m3_data["total_3"]), Fs=fs, detrend="mean", vmin=-110)
# ax.set(xlabel=f"time since {begin_time.iso[11:19]} [s]", ylabel="Channel 3\nFreq [hz]", yticks=np.arange(25,125,25))


# plt.subplots_adjust(hspace=0.1)
